In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import pandas as pd

import json
from sentence_transformers import SentenceTransformer
import gzip
import os
import torch

In [2]:
# Read the data from a CSV file and select relevant columns
df_queries = pd.read_csv('antique_query_test.csv')
df_queries = df_queries[['query_id','text']]

df_docs = pd.read_csv('antique_sample_404k.csv')
df_docs = df_docs[['doc_id','text']]

df_qrel = pd.read_csv('antique_qurel_test.csv')
df_qrel = df_qrel[['query_id','doc_id','relevance']]

In [3]:
# Merge the relevance data with the document data based on doc_id
merged_df = df_qrel.merge(df_docs, on='doc_id', how='left')

In [4]:
# Display information about the merged DataFrame
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6589 entries, 0 to 6588
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   query_id   6589 non-null   int64 
 1   doc_id     6589 non-null   object
 2   relevance  6589 non-null   int64 
 3   text       6589 non-null   object
dtypes: int64(2), object(2)
memory usage: 206.0+ KB


In [5]:
# Extract the 'doc_id' and 'text' columns from the merged DataFrame
df_text = merged_df[['doc_id','text']]

# Initialize an empty list to store passages
passages = []

# Iterate through each row in the 'df_text' DataFrame and append text to the 'passages' list
for index, row in df_text.iterrows():
    passages.append(str(row['text']))

In [6]:
# Display the first 5 passages
passages[0:5]

['Weed could mean the bad thing that grow in ur graden or back and front yard. Or it could mean the drug.',
 'sell weed',
 'My weed!!',
 'because we dont know what the hell to make legal in the US anymore....i mean we still have Bush in office, you would think that legalizing weed would be less harsh of the 2',
 'Its a weed.']

In [7]:
# Initialize a SentenceTransformer model for embedding text
bi_encoder = SentenceTransformer('intfloat/e5-base-v2')
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
# top_k = 32                          #Number of passages we want to retrieve with the bi-encoder

In [8]:
# Encode the passages to obtain text embeddings
corpus_embeddings_text = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/206 [00:00<?, ?it/s]

In [9]:
# Save the corpus embeddings to a pickle file
import pickle
with open('corpus_embeddings_text.pickle', 'wb') as pkl:
    pickle.dump(corpus_embeddings_text, pkl)

In [10]:
# Encode the passages to obtain querey text embeddings
query_embeddings = bi_encoder.encode(df_queries['text'].tolist(), convert_to_tensor=True, show_progress_bar=True)


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [11]:
# Save the query embeddings to a pickle file
import pickle
with open('query_embeddings.pickle', 'wb') as pkl:
    pickle.dump(query_embeddings, pkl)